This is a follow-up of my [[Train] COVID-19 Detection using YOLOv5](https://www.kaggle.com/ayuraj/train-covid-19-detection-using-yolov5) kernel that will:

* Train YOLOv5 in a cross validation setting (It's just repeating the same thing 5 times in case of 5-fold training). 

* Shows **ensembling** of different YOLOv5 models to get ensembled results. 

# ☀️ Imports and Setup

According to the official [Train Custom Data](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data) guide, YOLOv5 requires a certain directory structure. 

```
/parent_folder
    /dataset
         /images
         /labels
    /yolov5
```

* We thus will create a `/tmp` directory. <br>
* Download YOLOv5 repository and pip install the required dependencies. <br>
* Install the latest version of W&B and login with your wandb account. You can create your free W&B account [here](https://wandb.ai/site).

In [ ]:
%cd ../
!mkdir tmp
%cd tmp

In [ ]:
# Download YOLOv5
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
# Install dependencies
%pip install -qr requirements.txt  # install dependencies

%cd ../
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
# Install W&B 
!pip install -q --upgrade wandb
# Login 
import wandb
wandb.login()

In [ ]:
# Necessary/extra dependencies. 
import os
import gc
import cv2 
import wandb
import shutil
import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment', None)
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

# 🦆 Hyperparameters

In [ ]:
TRAIN_PATH = '../input/siim-covid19-resized-to-256px-png/train'
IMG_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 100
USE_FOLD = False
SEED = 42
NUM_FOLD = 5

# 🔨 Prepare Dataset

This is the most important section when it comes to training an object detector with YOLOv5. The directory structure, bounding box format, etc must be in the correct order. This section builds every piece needed to train a YOLOv5 model.

I am using [xhlulu's](https://www.kaggle.com/xhlulu) resized dataset. The uploaded 256x256 Kaggle dataset is [here](https://www.kaggle.com/xhlulu/siim-covid19-resized-to-256px-jpg). Find other image resolutions [here](https://www.kaggle.com/c/siim-covid19-detection/discussion/239918).

* Create train-validation split. <br>
* Create required `/dataset` folder structure and more the images to that folder. <br>
* Create `data.yaml` file needed to train the model. <br>
* Create bounding box coordinates in the required YOLO format. 

### Merge Study and Image level CSV files

In [ ]:
# IMAGE LEVEL

# Load image level csv file
df = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
# Load study level csv file
label_df = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')

# Modify values in the id column
df['id'] = df.apply(lambda row: row.id.split('_')[0], axis=1)
# Add absolute path
# df['path'] = df.apply(lambda row: TRAIN_PATH+row.id+'.jpg', axis=1)
# Get image level labels
def image_level(row):
    label = row.label.split(' ')[0]
    if label == 'opacity': return 1
    else: return 0

df['image_level'] = df.apply(lambda row: image_level(row), axis=1)

# STUDY LEVEL

# Modify values in the id column
label_df['id'] = label_df.apply(lambda row: row.id.split('_')[0], axis=1)
# Rename the column id with StudyInstanceUID
label_df.columns = ['StudyInstanceUID', 'Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']

# Label encode study-level labels
labels = label_df[['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']].values
labels = np.argmax(labels, axis=1)
label_df['study_level'] = labels

# ORIGINAL DIMENSION

# Load meta.csv file
meta_df = pd.read_csv('../input/siim-covid19-resized-to-256px-png/meta.csv')
train_meta_df = meta_df.loc[meta_df.split == 'train']
train_meta_df = train_meta_df.drop('split', axis=1)
train_meta_df.columns = ['id', 'dim0', 'dim1']

# Merge image-level and study-level
df = df.merge(label_df, on='StudyInstanceUID',how="left")
# Merge with meta_df
df = df.merge(train_meta_df, on='id',how="left")

# Write as csv file
df.to_csv('_image_study_total.csv', index=False)

df.head(10)

In [ ]:
if 'train_fold.csv' in os.listdir(os.getcwd()) and USE_FOLD:
    df = pd.read_csv('train_fold.csv')
else:
    df = pd.read_csv('_image_study_total.csv')
    df['path'] = df.apply(lambda row: f'../input/siim-covid19-resized-to-256px-png/train/{row.id}.png', axis=1)
    
    # Group by Study Ids and remove images that are "assumed" to be mislabeled
    for grp_df in df.groupby('StudyInstanceUID'):
        grp_id, grp_df = grp_df[0], grp_df[1]
        if len(grp_df) == 1:
            pass
        else:
            for i in range(len(grp_df)):
                row = grp_df.loc[grp_df.index.values[i]]
                if row.study_level > 0 and row.boxes is np.nan:
                    df = df.drop(grp_df.index.values[i])
                    
    print('total number of images: ', len(df))
    
    # Create train and validation split.
    df = df.drop('boxes', axis=1).reset_index()
    Fold = StratifiedKFold(n_splits=NUM_FOLD, shuffle=True, random_state=SEED)
    for n, (train_index, val_index) in enumerate(Fold.split(df, df['image_level'])):
        df.loc[val_index, 'fold'] = int(n)
    df['fold'] = df['fold'].astype(int)

    df.to_csv('train_fold.csv', index=False)

df.head()

## 🍚 Prepare Required Folder Structure

The required folder structure for the dataset directory is: 

```
/parent_folder
    /dataset
         /images
             /train
             /val
         /labels
             /train
             /val
    /yolov5
```

Note that I have named the directory `covid`.

In [ ]:
!pwd

In [ ]:
if USE_FOLD:
    pass
else:
    # Remove existing dirs
    for fold in range(NUM_FOLD):
        # Prepare train and valid df
        train_df = df.loc[df.fold != fold].reset_index(drop=True)
        valid_df = df.loc[df.fold == fold].reset_index(drop=True)
        
        try:
            shutil.rmtree(f'dataset_folds_{fold}/images')
            shutil.rmtree(f'dataset_folds_{fold}/labels')
        except:
            print('No dirs')

        # Make new dirs
        os.makedirs(f'dataset_folds_{fold}/images/train', exist_ok=True)
        os.makedirs(f'dataset_folds_{fold}/images/valid', exist_ok=True)
        os.makedirs(f'dataset_folds_{fold}/labels/train', exist_ok=True)
        os.makedirs(f'dataset_folds_{fold}/labels/valid', exist_ok=True)

        # Move the images to relevant split folder.
        for i in tqdm(range(len(train_df))):
            row = train_df.loc[i]
            copyfile(row.path, f'dataset_folds_{fold}/images/train/{row.id}.png')
            
        for i in tqdm(range(len(valid_df))):
            row = valid_df.loc[i]
            copyfile(row.path, f'dataset_folds_{fold}/images/valid/{row.id}.png')

In [ ]:
!ls

## 🍜 Create `.YAML` file

The `data.yaml`, is the dataset configuration file that defines 

1. an "optional" download command/URL for auto-downloading, 
2. a path to a directory of training images (or path to a *.txt file with a list of training images), 
3. a path to a directory of validation images (or path to a *.txt file with a list of validation images), 
4. the number of classes, 
5. a list of class names.

> 📍 Important: In this competition, each image can either belong to `opacity` or `none` image-level labels. That's why I have  used the number of classes, `nc` to be 2. YOLOv5 automatically handles the images without any bounding box coordinates. 

> 📍 Note: The `data.yaml` is created in the `yolov5/data` directory as required. 

In [ ]:
# Create .yaml file 
import yaml

for fold in range(NUM_FOLD):
    data_yaml = dict(
        train = f'../dataset_folds_{fold}/images/train',
        val = f'../dataset_folds_{fold}/images/valid',
        nc = 2,
        names = ['none', 'opacity']
    )

    # Note that I am creating the file in the yolov5/data/ directory.
    with open(f'yolov5/data/data_fold_{fold}.yaml', 'w') as outfile:
        yaml.dump(data_yaml, outfile, default_flow_style=True)
    
%cat yolov5/data/data_fold_0.yaml

## 🍮 Prepare Bounding Box Coordinated for YOLOv5

For every image with **bounding box(es)** a `.txt` file with the same name as the image will be created in the format shown below:

* One row per object. <br>
* Each row is class `x_center y_center width height format`. <br>
* Box coordinates must be in normalized xywh format (from 0 - 1). We can normalize by the boxes in pixels by dividing `x_center` and `width` by image width, and `y_center` and `height` by image height. <br>
* Class numbers are zero-indexed (start from 0). <br>

> 📍 Note: We don't have to remove the images without bounding boxes from the training or validation sets. 

In [ ]:
# Get the raw bounding box by parsing the row value of the label column.
# Ref: https://www.kaggle.com/yujiariyasu/plot-3positive-classes
def get_bbox(row):
    bboxes = []
    bbox = []
    for i, l in enumerate(row.label.split(' ')):
        if (i % 6 == 0) | (i % 6 == 1):
            continue
        bbox.append(float(l))
        if i % 6 == 5:
            bboxes.append(bbox)
            bbox = []  
            
    return bboxes

# Scale the bounding boxes according to the size of the resized image. 
def scale_bbox(row, bboxes):
    # Get scaling factor
    scale_x = IMG_SIZE/row.dim1
    scale_y = IMG_SIZE/row.dim0
    
    scaled_bboxes = []
    for bbox in bboxes:
        x = int(np.round(bbox[0]*scale_x, 4))
        y = int(np.round(bbox[1]*scale_y, 4))
        x1 = int(np.round(bbox[2]*(scale_x), 4))
        y1= int(np.round(bbox[3]*scale_y, 4))

        scaled_bboxes.append([x, y, x1, y1]) # xmin, ymin, xmax, ymax
        
    return scaled_bboxes

# Convert the bounding boxes in YOLO format.
def get_yolo_format_bbox(img_w, img_h, bboxes):
    yolo_boxes = []
    for bbox in bboxes:
        w = bbox[2] - bbox[0] # xmax - xmin
        h = bbox[3] - bbox[1] # ymax - ymin
        xc = bbox[0] + int(np.round(w/2)) # xmin + width/2
        yc = bbox[1] + int(np.round(h/2)) # ymin + height/2
        
        yolo_boxes.append([xc/img_w, yc/img_h, w/img_w, h/img_h]) # x_center y_center width height
    
    return yolo_boxes

In [ ]:
def write_bbox_files(tmp_df, fold_num, split):
    path = f'dataset_folds_{fold}/labels/{split}'
    for i in tqdm(range(len(tmp_df))):
        row = tmp_df.loc[i]
        # Get image id
        img_id = row.id
        # Get image-level label
        label = row.image_level

        file_name = f'{path}/{img_id}.txt'

        if label==1:
            # Get bboxes
            bboxes = get_bbox(row)
            # Scale bounding boxes
            scale_bboxes = scale_bbox(row, bboxes)
            # Format for YOLOv5
            yolo_bboxes = get_yolo_format_bbox(IMG_SIZE, IMG_SIZE, scale_bboxes)

            with open(file_name, 'w') as f:
                for bbox in yolo_bboxes:
                    bbox = [1]+bbox
                    bbox = [str(i) for i in bbox]
                    bbox = ' '.join(bbox)
                    f.write(bbox)
                    f.write('\n')

if USE_FOLD:
    pass
else:
    # Prepare the txt files for bounding box
    for fold in range(NUM_FOLD):
        # Prepare train and valid df
        train_df = df.loc[df.fold != fold].reset_index(drop=True)
        valid_df = df.loc[df.fold == fold].reset_index(drop=True)
        
        # prepare label for train
        write_bbox_files(train_df, fold, 'train')
        # prepare label for valid
        write_bbox_files(valid_df, fold, 'valid')

# 🚅 5 Fold Training with W&B



In [ ]:
%cd yolov5

```
--img {IMG_SIZE} \ # Input image size.
--batch {BATCH_SIZE} \ # Batch size
--epochs {EPOCHS} \ # Number of epochs
--data data.yaml \ # Configuration file
--weights yolov5s.pt \ # Model name
--save_period 1\ # Save model after interval
--project kaggle-siim-covid # W&B project name
```

In [ ]:
for fold in range(NUM_FOLD):    
    print('FOLD NUMBER: ', fold)
    !python train.py --img {IMG_SIZE} \
                     --batch {BATCH_SIZE} \
                     --epochs {1} \
                     --data data_fold_{fold}.yaml \
                     --weights yolov5s.pt \
                     --save_period 10\
                     --project yolov5-covid19-folds\
                     --name yolov5s-e-100-img-256-fold-{fold}
    print('###########################################################################################\n')

I have trained YOLOv5 small using image resolution of 512x512 and 5 fold stratified split of the dataset. The models were trained using single V100 GPU. The GIF below shows the training metrics logged on the W&B dashboard. 

The model weights can be found [here](https://www.kaggle.com/ayuraj/yolo-models).

### [Check out the W&B Dashboard](https://wandb.ai/ayush-thakur/kaggle-covid19-folds)

![img](https://i.imgur.com/MlNRJgU.gif) 

# ✈️ Get OOF Predictions

In [ ]:
!ls yolov5-covid19-folds

In [ ]:
MODEL_WEIGHTS = [
    'yolov5-covid19-folds/yolov5s-e-100-img-256-fold-0/weights/best.pt',
    'yolov5-covid19-folds/yolov5s-e-100-img-256-fold-1/weights/best.pt',
    'yolov5-covid19-folds/yolov5s-e-100-img-256-fold-2/weights/best.pt',
    'yolov5-covid19-folds/yolov5s-e-100-img-256-fold-3/weights/best.pt',
    'yolov5-covid19-folds/yolov5s-e-100-img-256-fold-4/weights/best.pt',
]

SOURCES = [
    '../dataset_folds_0/images/valid',
    '../dataset_folds_1/images/valid',
    '../dataset_folds_2/images/valid',
    '../dataset_folds_3/images/valid',
    '../dataset_folds_4/images/valid',
]

CONFIDENCE = [
    0.269, 0.268, 0.209, 0.179, 0.308
]

In [ ]:
for fold in range(NUM_FOLD):
    print('FOLD NUMBER: ', fold)
    
    !python detect.py --weights {MODEL_WEIGHTS[fold]} \
                      --source {SOURCES[fold]} \
                      --img {IMG_SIZE} \
                      --conf {CONFIDENCE[fold]} \
                      --iou-thres 0.5 \
                      --max-det 3 \
                      --name infer_fold_{fold}\
                      --save-txt \
                      --save-conf\
                      --nosave
    
    print('###########################################################################################\n')    

# 🚀 Ensembling

Ensembling YOLOv5 models is as easy as calling `python detect.py` and pass in model weights as arguments:

`python detect.py --weights yolov5x.pt yolov5l6.pt --img 640 --source data/images`

You can learn more about ensembling [here](https://github.com/ultralytics/yolov5/issues/318).

### You can check out my [Submission Covid19](https://www.kaggle.com/ayuraj/submission-covid19) kernel for Ensembling. 